In [9]:
import glob

path = "../api/index_documents"

files  = glob.glob(path + "/*")

#for filename in glob.glob(f"{path}/*"):
#    print(filename)

files

['../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf',
 '../api/index_documents\\UH Onboarding Manual.pdf']

In [12]:
from langchain_community.document_loaders import PyPDFLoader

docs = []
for file in files:
    loader = PyPDFLoader(file)
    pages = loader.load()
    docs.append(pages)
    print(file)

../api/index_documents\ED2_PAPER+ID+05+GENERATIVE+ART.pdf
../api/index_documents\UH Onboarding Manual.pdf


In [27]:
for doc in docs:
    for d in doc:
        print(d.page_content)

[58] 
 
 
 
UNDERSTANDING OF GENERATIVE ART WITH ‘PYTHON’ 
PROGRAMMING 
 
 
Teddy Marius Soikun1* 
 
1Academy of Arts and Creative Technology,  
University Malaysia Sabah, 88400 Kota Kinabalu, Malaysia  
*tsoikun@ums.edu.my 
 
Received: 21 June 2023 | Revised: 10 July 2023  
Accepted: 30 September 2023 / Published: 20 December 2023 
DOI: https://10.51200/ga.v13i2.4735     
 
 
ABSTRACT 
The article uses the Python programming language to introduce generative arts as an alternative 
to conventional art and design. Understanding generative arts in the age of Artificial 
Intelligence (AI) is seen as a way forward to remain relevant. Through the use of a practice -
based research methodology, the article explores principles, processes, and tools for generating 
art through coding. Practical samples are used to demonstrate the potential of generative art 
through basic coding examples. This research also highlights the role of generative art in 
fostering interdisciplinary collaboration and

In [29]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document
import os

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_key=os.getenv('AZURE_OPENAI_KEY'),
                azure_deployment=os.getenv('EMBEDDINGS_DEPLOYMENT_NAME')
            )



vector_store = FAISS.from_documents(docs[0], embeddings)

#vector_store.save_local(os.path.join(full_path, "faiss_index"))
#index_path = os.path.join(full_path, "faiss_index")

In [36]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

d = 1536
faiss_index = faiss.IndexFlatL2(d)

vs = FAISS( 
    embedding_function=embeddings,
    index=faiss_index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
    )

In [37]:
vs.add_documents(docs[0])

['150c06d2-c2e2-4606-b532-f2d23d9b8a69',
 'f9a3bcd5-8cb8-44e3-b608-4c98296e073f',
 '527f2c0c-b259-49dc-8508-9aa38a698674',
 '5f87d2b1-b797-4e96-822c-0dd3e08603a9',
 '1dba8287-2d01-4383-84ed-fec70cd962f8',
 '476f9526-aa71-48e9-bc16-df5a9980763e',
 '52aae229-4cbf-4fd5-bfa4-bea8ac2f55d7',
 '6123d1dc-99ef-4ab5-bdf1-9400f813af63',
 'b3b1e494-4f60-4adb-8c05-d390e5b19865',
 '51cf03b7-da8a-4047-b076-1ad8ce488ef3',
 '35afd121-e91c-422b-8e3b-bad72cb871cc',
 '66c101ef-485b-4ccd-bd36-e21178638350']

In [30]:
vector_store.similarity_search("genai", k=3)

[Document(id='98c139b5-e527-497c-a3dd-eabc9c2f63c0', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-12-15T20:26:26+08:00', 'author': 'Teddy Soikun', 'moddate': '2023-12-15T20:26:26+08:00', 'source': '../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf', 'total_pages': 12, 'page': 11, 'page_label': '12'}, page_content='[69] \n \nREFERENCES \n \nA., A. (2023). The Renaissance of Creativity: Exploring the Impact of Generative AI in the Creative \nIndustry. https://www.linkedin.com/pulse/renaissance -creativity-exploring-impact-generative-ai-\nadam \nAela, E. (2023, April 1). Artificial Intelligence: How AI is Changing Art. \nhttps://aelaschool.com/en/art/artificial-intelligence-art-changes/ \nAhmed, R. (2023). The Future of Art and Creativity in the Age of AI. \nhttps://www.linkedin.com/pulse/future-art-creativity-age-ai-rohan-ahmed \nGwafan, D. (2023). Python Programming Language: A Versatile and Powerful Tool for Modern \nDe

In [31]:
vector_store.add_documents(docs[1])

['fa156ecb-4c89-494c-8b0d-3d38901e0ae7',
 'ea4f3545-a1e2-4ab8-b993-b355e2b0010e',
 'e16f5a8a-51b7-43bd-9c95-84a648f24d66',
 '8b1782b8-683f-4df4-ae78-f3b8801ddfc1',
 '7ec44bd8-504a-4cb6-9030-649a3e984edb',
 'fc327a2e-c2e1-4549-8577-4444b31151c5',
 '5d3aa8b5-b901-4641-88cb-7578e4e6f172',
 '8a2fc980-8225-46f0-af16-602d55ae831c',
 '39dea5bf-533e-4812-a128-7c818b2a17f1',
 '8f1f1ae9-ce66-450a-a390-735925bc2077',
 '85948427-e677-4ce7-aecb-eb560431e535',
 '298592f0-fd94-4109-b41d-4f22f9cfef43',
 '3516f86b-7e3c-44cf-b605-6cda18785949',
 '26f3cc09-1548-47aa-9270-65437519ebe8',
 '8adf4f99-ddcf-485f-9a48-ba939c62f278',
 '10356fe9-a477-4186-8fcc-3e6378872210',
 '80372780-e92f-4f46-8a21-2dfda4eda55d',
 '3c88c04a-4ba5-4fd0-bb21-25af403bf9e5']

In [33]:
vector_store.similarity_search("what to do in first week", k=3)

[Document(id='7ec44bd8-504a-4cb6-9030-649a3e984edb', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2018-11-08T10:22:04-06:00', 'author': 'Waggoner, Ashley', 'moddate': '2018-11-08T11:27:10-06:00', 'title': 'Onboarding Toolkit', 'source': '../api/index_documents\\UH Onboarding Manual.pdf', 'total_pages': 18, 'page': 4, 'page_label': '5'}, page_content='Page | 3 \n \n\uf07f Benefits Information \n\uf07f UH ID \n\uf07f Parking permit \n \nDAY 1 - MANAGER \n\uf07f When possible, escort your new employee from NEC to your office (Or send representative). \n\uf07f Send out welcome email to department \n\uf07f If the employee is bi-weekly, discuss TRAM procedures \n\uf07f Answer any immediate questions the employee may have \n\uf07f Tour of department & introductions to co-workers \n\uf07f Provide keys, uniform, equipment, etc \n\uf07f Give employee time to get acquainted with his/her workspace \no Log into computer/phone systems \no Set up e